# NYC Food Rescue Stochastic Optimization - Visualizations

Visualizations for the two-stage stochastic model:
- **First stage**: Which distribution centers to open
- **Second stage**: Flow decisions under demand scenarios (2023, 2024, 2025)

In [16]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import folium
from folium.plugins import MarkerCluster
from IPython.display import display, HTML

## Load Data

In [17]:
pareto_df = pd.read_csv('viz_data/pareto_results.csv')
restaurants_df = pd.read_csv('viz_data/restaurants.csv')
centers_df = pd.read_csv('viz_data/donation_centers.csv')
neighborhoods_df = pd.read_csv('viz_data/neighborhoods.csv')
scenario_summary = pd.read_csv('viz_data/scenario_summary.csv')

try:
    centers_opened = pd.read_csv('viz_data/centers_opened.csv')
    flows_df = pd.read_csv('viz_data/flows_2023.csv')
    alloc_2023 = pd.read_csv('viz_data/allocations_2023.csv')
    alloc_2024 = pd.read_csv('viz_data/allocations_2024.csv')
    alloc_2025 = pd.read_csv('viz_data/allocations_2025.csv')
except:
    centers_opened, flows_df = None, None
    alloc_2023, alloc_2024, alloc_2025 = None, None, None

print(f"Pareto solutions: {len(pareto_df)}")
print(f"Restaurants: {len(restaurants_df)}")
print(f"Centers: {len(centers_df)}")
print(f"Neighborhoods: {len(neighborhoods_df)}")

Pareto solutions: 110
Restaurants: 319
Centers: 201
Neighborhoods: 197


## Color Theme

In [18]:
COLORS = {
    'background': '#0a0a0f',
    'surface': '#14141f',
    'primary': '#ff6b35',
    'secondary': '#4ecdc4',
    'accent': '#ffe66d',
    'purple': '#a855f7',
    'text': '#f7f7f7',
    'text_muted': '#8b8b9e',
    'grid': '#2a2a3e',
    'scenario_2023': '#ff6b35',
    'scenario_2024': '#4ecdc4',
    'scenario_2025': '#a855f7'
}

# General Map of the distribution centers, restaurants and neighborhood in NYC

In [19]:
# Create map
m = folium.Map(location=[40.7128, -73.9560], zoom_start=11, tiles='CartoDB positron')

# Restaurants (orange circles, clustered)
restaurant_cluster = MarkerCluster(name='Restaurants (Supply)')
for _, row in restaurants_df.iterrows():
    folium.CircleMarker(
        location=[row['latitude'], row['longitude']],
        radius=5,
        color='#ff6b35',
        fill=True,
        fill_color='#ff6b35',
        fill_opacity=0.8,
        popup=f"Restaurant {int(row['id'])}<br>Supply: {row['supply']:,.0f} lbs"
    ).add_to(restaurant_cluster)
restaurant_cluster.add_to(m)

# Distribution Centers (green squares)
for _, row in centers_df.iterrows():
    folium.RegularPolygonMarker(
        location=[row['latitude'], row['longitude']],
        number_of_sides=4,
        radius=8,
        color='#2ecc71',
        fill=True,
        fill_color='#2ecc71',
        fill_opacity=0.9,
        popup=f"Distribution Center {int(row['id'])}"
    ).add_to(m)

# Neighborhoods (purple circles, sized by demand)
demand_col = 'demand_2023' if 'demand_2023' in neighborhoods_df.columns else 'demand'
demand_nbhd = neighborhoods_df[neighborhoods_df[demand_col] > 0]
max_demand = demand_nbhd[demand_col].max()

for _, row in demand_nbhd.iterrows():
    radius = 4 + 12 * (row[demand_col] / max_demand)
    folium.CircleMarker(
        location=[row['latitude'], row['longitude']],
        radius=radius,
        color='#9b59b6',
        fill=True,
        fill_color='#9b59b6',
        fill_opacity=0.5,
        popup=f"Neighborhood {int(row['id'])}<br>Demand: {row[demand_col]:,.0f} lbs"
    ).add_to(m)

# Add legend
legend_html = '''
<div style="position: fixed; 
            bottom: 30px; left: 30px; 
            background-color: white;
            border: 2px solid #333;
            border-radius: 8px;
            padding: 12px 16px;
            z-index: 9999;
            font-family: Arial, sans-serif;
            font-size: 13px;
            box-shadow: 2px 2px 6px rgba(0,0,0,0.3);">
    <b style="font-size: 14px;">Legend</b><br><br>
    <span style="color: #ff6b35;">●</span> Restaurants (Supply)<br>
    <span style="color: #2ecc71;">■</span> Distribution Centers<br>
    <span style="color: #9b59b6;">●</span> Neighborhoods (Demand)
</div>
'''
m.get_root().html.add_child(folium.Element(legend_html))

# Add layer control
folium.LayerControl().add_to(m)

# Save
m.save('figure1_network_locations.html')
print('Saved: figure1_network_locations.html')
m

Saved: figure1_network_locations.html


---
## 1. Pareto Frontier (Cost vs. Equity)

In [20]:
df = pareto_df.sort_values('avg_transport_cost').copy()

fig = go.Figure()

# Pareto line
fig.add_trace(go.Scatter(
    x=df['avg_transport_cost'], y=df['avg_equity_t'],
    mode='lines',
    line=dict(color=COLORS['primary'], width=3, shape='spline', smoothing=0.8),
    hoverinfo='skip',
    name='Pareto Frontier'
))

# Points with hover info
hover_text = [f"<b>w_cost={row.w_cost:.1f}, w_eq={row.w_eq:.1f}</b><br>" +
              f"Avg Transport Cost: {row.avg_transport_cost:,.0f}<br>" +
              f"Avg Worst Unmet: {row.avg_equity_t:,.0f}<br>" +
              f"Centers Opened: {row.num_centers}<br>" +
              f"Avg Delivered: {row.avg_total_recv:,.0f}"
              for _, row in df.iterrows()]

fig.add_trace(go.Scatter(
    x=df['avg_transport_cost'], y=df['avg_equity_t'],
    mode='markers',
    marker=dict(
        size=10,
        color=df['num_centers'],
        colorscale='Viridis',
        colorbar=dict(title='Centers<br>Opened'),
        line=dict(color='white', width=1)
    ),
    text=hover_text,
    hovertemplate='%{text}<extra></extra>',
    name='Solutions'
))

# Annotations for extremes
cost_opt = df.loc[df['avg_transport_cost'].idxmin()]
eq_opt = df.loc[df['avg_equity_t'].idxmin()]

fig.add_annotation(x=cost_opt['avg_transport_cost'], y=cost_opt['avg_equity_t'],
                   text="Cost Optimal", showarrow=True, arrowcolor=COLORS['secondary'],
                   font=dict(color=COLORS['secondary'], size=11), ax=-60, ay=-30)
fig.add_annotation(x=eq_opt['avg_transport_cost'], y=eq_opt['avg_equity_t'],
                   text="Equity Optimal", showarrow=True, arrowcolor=COLORS['accent'],
                   font=dict(color=COLORS['accent'], size=11), ax=60, ay=30)

fig.update_layout(
    title=dict(text="<b>Pareto Frontier: Cost vs. Equity</b><br><sub>Stochastic Model (3 Demand Scenarios)</sub>", 
               x=0.5, font=dict(size=18, color=COLORS['text'])),
    xaxis=dict(title="Avg. Transportation Cost", tickformat=',', gridcolor=COLORS['grid'], color=COLORS['text_muted']),
    yaxis=dict(title="Avg. Worst Unmet Demand", tickformat=',', gridcolor=COLORS['grid'], color=COLORS['text_muted']),
    plot_bgcolor=COLORS['surface'],
    paper_bgcolor=COLORS['background'],
    font=dict(color=COLORS['text']),
    showlegend=False,
    height=550
)

fig.show()

---
## 2. Scenario Comparison: Worst Unmet Demand by Year

In [21]:
# Select a few representative solutions
df_sorted = pareto_df.sort_values('avg_equity_t')
selected_idx = [0, len(df_sorted)//4, len(df_sorted)//2, 3*len(df_sorted)//4, len(df_sorted)-1]
selected = df_sorted.iloc[selected_idx].reset_index(drop=True)

fig = go.Figure()

x_labels = [f"w_c={row.w_cost:.1f}\nw_e={row.w_eq:.1f}" for _, row in selected.iterrows()]

fig.add_trace(go.Bar(name='2023', x=x_labels, y=selected['t_2023'], marker_color=COLORS['scenario_2023']))
fig.add_trace(go.Bar(name='2024', x=x_labels, y=selected['t_2024'], marker_color=COLORS['scenario_2024']))
fig.add_trace(go.Bar(name='2025', x=x_labels, y=selected['t_2025'], marker_color=COLORS['scenario_2025']))

fig.update_layout(
    title=dict(text="<b>Worst Unmet Demand by Scenario</b>", x=0.5, font=dict(size=16, color=COLORS['text'])),
    xaxis=dict(title="Solution Weights", color=COLORS['text_muted']),
    yaxis=dict(title="Worst Unmet Demand (lbs)", tickformat=',', gridcolor=COLORS['grid'], color=COLORS['text_muted']),
    barmode='group',
    plot_bgcolor=COLORS['surface'],
    paper_bgcolor=COLORS['background'],
    font=dict(color=COLORS['text']),
    legend=dict(title='Scenario', orientation='h', y=1.1, x=0.5, xanchor='center'),
    height=450
)

fig.show()

---
## 3. Centers Opened vs. Objective Trade-off

In [22]:
fig = make_subplots(rows=1, cols=2, subplot_titles=('Centers vs. Transport Cost', 'Centers vs. Equity'))

fig.add_trace(go.Scatter(
    x=pareto_df['num_centers'], y=pareto_df['avg_transport_cost'],
    mode='markers',
    marker=dict(size=10, color=COLORS['primary'], opacity=0.7),
    hovertemplate='Centers: %{x}<br>Cost: %{y:,.0f}<extra></extra>'
), row=1, col=1)

fig.add_trace(go.Scatter(
    x=pareto_df['num_centers'], y=pareto_df['avg_equity_t'],
    mode='markers',
    marker=dict(size=10, color=COLORS['secondary'], opacity=0.7),
    hovertemplate='Centers: %{x}<br>Worst Unmet: %{y:,.0f}<extra></extra>'
), row=1, col=2)

fig.update_layout(
    title=dict(text="<b>Impact of Number of Centers Opened</b>", x=0.5, font=dict(size=16, color=COLORS['text'])),
    plot_bgcolor=COLORS['surface'],
    paper_bgcolor=COLORS['background'],
    font=dict(color=COLORS['text']),
    showlegend=False,
    height=400
)
fig.update_xaxes(title_text='Centers Opened', gridcolor=COLORS['grid'])
fig.update_yaxes(gridcolor=COLORS['grid'], tickformat=',')

fig.show()

---
## 4. Scenario Summary (Bar Chart)

In [23]:
fig = make_subplots(rows=1, cols=3, subplot_titles=('Total Demand', 'Total Received', 'Coverage %'))

colors = [COLORS['scenario_2023'], COLORS['scenario_2024'], COLORS['scenario_2025']]

fig.add_trace(go.Bar(
    x=scenario_summary['scenario'], y=scenario_summary['total_demand'],
    marker_color=colors, showlegend=False
), row=1, col=1)

fig.add_trace(go.Bar(
    x=scenario_summary['scenario'], y=scenario_summary['total_received'],
    marker_color=colors, showlegend=False
), row=1, col=2)

coverage = 100 * scenario_summary['total_received'] / scenario_summary['total_demand']
fig.add_trace(go.Bar(
    x=scenario_summary['scenario'], y=coverage,
    marker_color=colors, showlegend=False,
    text=[f"{v:.1f}%" for v in coverage], textposition='outside'
), row=1, col=3)

fig.update_layout(
    title=dict(text="<b>Scenario Performance Summary</b>", x=0.5, font=dict(size=16, color=COLORS['text'])),
    plot_bgcolor=COLORS['surface'],
    paper_bgcolor=COLORS['background'],
    font=dict(color=COLORS['text']),
    height=400
)
fig.update_yaxes(gridcolor=COLORS['grid'], tickformat=',')

fig.show()

---
## 5. Network Map with Opened Centers

In [24]:
m = folium.Map(location=[40.7128, -73.9560], zoom_start=11, tiles='CartoDB dark_matter')

# Restaurants (clustered)
restaurant_cluster = MarkerCluster(name='🍽️ Restaurants')
for _, row in restaurants_df.iterrows():
    folium.CircleMarker(
        location=[row['latitude'], row['longitude']],
        radius=4, color='#ff6b35', fill=True, fill_opacity=0.7,
        popup=f"Restaurant {int(row['id'])}<br>Supply: {row['supply']:,.0f}"
    ).add_to(restaurant_cluster)
restaurant_cluster.add_to(m)

# Centers (opened vs closed)
if centers_opened is not None:
    centers_merged = centers_df.merge(centers_opened, on='id')
    for _, row in centers_merged.iterrows():
        color = 'green' if row['opened'] == 1 else 'gray'
        icon = 'check' if row['opened'] == 1 else 'times'
        folium.Marker(
            location=[row['latitude'], row['longitude']],
            icon=folium.Icon(color=color, icon=icon, prefix='fa'),
            popup=f"Center {int(row['id'])}<br>{'OPEN' if row['opened']==1 else 'Closed'}"
        ).add_to(m)
else:
    for _, row in centers_df.iterrows():
        folium.Marker(
            location=[row['latitude'], row['longitude']],
            icon=folium.Icon(color='green', icon='archive', prefix='fa'),
            popup=f"Center {int(row['id'])}"
        ).add_to(m)

# Neighborhoods with demand (using 2023)
demand_col = 'demand_2023' if 'demand_2023' in neighborhoods_df.columns else 'demand'
demand_nbhd = neighborhoods_df[neighborhoods_df[demand_col] > 0]
max_demand = demand_nbhd[demand_col].max()

for _, row in demand_nbhd.iterrows():
    folium.CircleMarker(
        location=[row['latitude'], row['longitude']],
        radius=3 + 12 * (row[demand_col] / max_demand),
        color='#a855f7', fill=True, fill_opacity=0.5,
        popup=f"Neighborhood {int(row['id'])}<br>Demand 2023: {row[demand_col]:,.0f}"
    ).add_to(m)

# Flows
if flows_df is not None and len(flows_df) > 0:
    rest_coords = dict(zip(restaurants_df['id'], zip(restaurants_df['latitude'], restaurants_df['longitude'])))
    center_coords = dict(zip(centers_df['id'], zip(centers_df['latitude'], centers_df['longitude'])))
    nbhd_coords = dict(zip(neighborhoods_df['id'], zip(neighborhoods_df['latitude'], neighborhoods_df['longitude'])))
    
    sig_flows = flows_df[flows_df['flow'] > 1000]
    max_flow = sig_flows['flow'].max() if len(sig_flows) > 0 else 1
    
    for _, row in sig_flows.iterrows():
        from_coord = rest_coords.get(row['from_id']) if row['from_type'] == 'restaurant' else center_coords.get(row['from_id'])
        to_coord = center_coords.get(row['to_id']) if row['to_type'] == 'center' else nbhd_coords.get(row['to_id'])
        if from_coord and to_coord:
            folium.PolyLine(
                [from_coord, to_coord],
                weight=1 + 3 * (row['flow'] / max_flow),
                color='#ffe66d', opacity=0.6
            ).add_to(m)

folium.LayerControl().add_to(m)
m.save('network_map.html')
print('Saved: network_map.html')
m

Saved: network_map.html


---
## 6. Demand Heatmap

In [25]:
demand_col = 'demand_2023' if 'demand_2023' in neighborhoods_df.columns else 'demand'

fig = go.Figure(go.Densitymap(
    lat=neighborhoods_df['latitude'],
    lon=neighborhoods_df['longitude'],
    z=neighborhoods_df[demand_col],
    radius=20,
    colorscale=[
        [0, 'rgba(78, 205, 196, 0.1)'],
        [0.5, 'rgba(255, 107, 53, 0.6)'],
        [1, 'rgba(168, 85, 247, 1)']
    ],
    colorbar=dict(title=dict(text='Demand (lbs)'))
))

fig.update_layout(
    map=dict(style='carto-darkmatter', center=dict(lat=40.7128, lon=-73.9560), zoom=10),
    title=dict(text="<b>Food Demand Heatmap (2023)</b>", x=0.5, font=dict(size=16, color='white')),
    paper_bgcolor='#0a0a0f',
    margin=dict(l=0, r=0, t=50, b=0),
    height=500
)

fig.show()

---
## 7. Allocation Distribution by Scenario

In [26]:
if alloc_2023 is not None:
    fig = go.Figure()
    
    fig.add_trace(go.Histogram(x=alloc_2023['received'], name='2023', marker_color=COLORS['scenario_2023'], opacity=0.7))
    fig.add_trace(go.Histogram(x=alloc_2024['received'], name='2024', marker_color=COLORS['scenario_2024'], opacity=0.7))
    fig.add_trace(go.Histogram(x=alloc_2025['received'], name='2025', marker_color=COLORS['scenario_2025'], opacity=0.7))
    
    fig.update_layout(
        title=dict(text="<b>Distribution of Food Received by Neighborhood</b>", x=0.5, font=dict(size=16, color=COLORS['text'])),
        xaxis=dict(title="Food Received (lbs)", gridcolor=COLORS['grid'], color=COLORS['text_muted']),
        yaxis=dict(title="Number of Neighborhoods", gridcolor=COLORS['grid'], color=COLORS['text_muted']),
        barmode='overlay',
        plot_bgcolor=COLORS['surface'],
        paper_bgcolor=COLORS['background'],
        font=dict(color=COLORS['text']),
        legend=dict(title='Scenario'),
        height=400
    )
    
    fig.show()
else:
    print("Allocation data not available. Run Julia export first.")

---
## 8. Unmet Demand Comparison Across Scenarios

In [27]:
if alloc_2023 is not None:
    fig = go.Figure()
    
    fig.add_trace(go.Box(y=alloc_2023['unmet'], name='2023', marker_color=COLORS['scenario_2023']))
    fig.add_trace(go.Box(y=alloc_2024['unmet'], name='2024', marker_color=COLORS['scenario_2024']))
    fig.add_trace(go.Box(y=alloc_2025['unmet'], name='2025', marker_color=COLORS['scenario_2025']))
    
    fig.update_layout(
        title=dict(text="<b>Unmet Demand Distribution by Scenario</b>", x=0.5, font=dict(size=16, color=COLORS['text'])),
        yaxis=dict(title="Unmet Demand (lbs)", gridcolor=COLORS['grid'], color=COLORS['text_muted'], tickformat=','),
        plot_bgcolor=COLORS['surface'],
        paper_bgcolor=COLORS['background'],
        font=dict(color=COLORS['text']),
        height=400
    )
    
    fig.show()
else:
    print("Allocation data not available. Run Julia export first.")

---
## 9. Summary Statistics

In [28]:
total_supply = restaurants_df['supply'].sum()
print("="*60)
print("STOCHASTIC OPTIMIZATION SUMMARY")
print("="*60)
print(f"\nNetwork Size:")
print(f"  Restaurants (supply nodes): {len(restaurants_df)}")
print(f"  Distribution Centers: {len(centers_df)}")
print(f"  Neighborhoods (demand nodes): {len(neighborhoods_df)}")
print(f"\nTotal Supply: {total_supply:,.0f} lbs")

if 'demand_2023' in neighborhoods_df.columns:
    print(f"\nDemand by Scenario:")
    print(f"  2023: {neighborhoods_df['demand_2023'].sum():,.0f} lbs")
    print(f"  2024: {neighborhoods_df['demand_2024'].sum():,.0f} lbs")
    print(f"  2025: {neighborhoods_df['demand_2025'].sum():,.0f} lbs")

print(f"\nPareto Solutions Generated: {len(pareto_df)}")
print(f"  Best Avg Transport Cost: {pareto_df['avg_transport_cost'].min():,.0f}")
print(f"  Best Avg Equity (min worst unmet): {pareto_df['avg_equity_t'].min():,.0f}")
print(f"  Centers Opened Range: {pareto_df['num_centers'].min()} - {pareto_df['num_centers'].max()}")

STOCHASTIC OPTIMIZATION SUMMARY

Network Size:
  Restaurants (supply nodes): 319
  Distribution Centers: 201
  Neighborhoods (demand nodes): 197

Total Supply: 16,034,410 lbs

Demand by Scenario:
  2023: 42,614,868 lbs
  2024: 59,065,198 lbs
  2025: 50,759,265 lbs

Pareto Solutions Generated: 110
  Best Avg Transport Cost: 819,913
  Best Avg Equity (min worst unmet): 2,676,561
  Centers Opened Range: 102 - 127
